# Integrate **EARLIEST** and **SYMBOL** in nakeds.py to make `lone.py`
* One symbol
* One dte
* All Nakeds fields recalculated

In [1]:
MARKET = 'NSE'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_nakeds.pkl', 'df_ohlcs.pkl',
       'df_opt_margins.pkl', 'df_opt_prices.pkl', 'df_opts.pkl',
       'df_symlots.pkl', 'df_unds.pkl', 'dfrq.pkl', 'qopt_rejects.pkl',
       'qopts.pkl'], dtype='<U18')

In [7]:
from nakeds import get_nakeds

In [18]:
RECALC_UNDS = True
SYMBOL = 'IBULHSGFI'
EARLIEST = True

In [19]:
df = get_nakeds(MARKET=MARKET, RECALC_UNDS=RECALC_UNDS, SYMBOL=SYMBOL, EARLIEST = EARLIEST, SAVE = False)


nakeds started at 07-Dec-2020 00:56:47


und:        0%|                         | 0/1 [00:00<?, ?it/s]                  


nse underlyings started at 07-Dec-2020 00:56:47



...nse underlyings took: 00:00:13 seconds


dfrqs started at 07-Dec-2020 00:57:01

...dfrqs took: 00:00:00 seconds



price:      0%|                         | 0/88 [00:00<?, ?it/s]                 


NSE nakeds option price started at 07-Dec-2020 00:57:01


margin:     0%|                         | 0/88 [00:00<?, ?it/s]                 


...NSE nakeds option price took: 00:00:17 seconds


NSE naked option margins started at 07-Dec-2020 00:57:19



...NSE naked option margins took: 00:00:06 seconds


...nakeds took: 00:00:37 seconds



In [20]:
# Filter for options with timevalue
m1 = ((df.right == 'C') & (df.strike > df.undPrice)) | \
    ((df.right == 'P') & (df.strike < df.undPrice))
df.drop(['contract', 'comm', 'hi_sd', 'lo_sd', 'secType', 'conId'], 1)[m1]

,symbol,expiry,strike,dte,right,und_iv,undPrice,lot,iv,qty,margin,bid,ask,close,last,sdMult,intrinsic,timevalue,price,expPrice,rom,expRom
2,IBULHSGFI,20201231,260.0,25,C,0.808505,193.3,3100.0,0.808505,3100.0,213370.37,-1.0,-1.0,4.75,NaN,1.630752,0.0,4.75,4.75,4.75,1.006199,1.006199
4,IBULHSGFI,20201231,265.0,25,C,0.808505,193.3,3100.0,0.808505,3100.0,205155.36,-1.0,-1.0,4.25,NaN,1.752997,0.0,4.25,4.25,4.25,0.936183,0.936183
5,IBULHSGFI,20201231,150.0,25,P,0.808505,193.3,3100.0,0.808505,3100.0,205758.17,-1.0,-1.0,2.50,NaN,1.058644,0.0,2.50,2.50,3.20,0.548498,0.700000
6,IBULHSGFI,20201231,145.0,25,P,0.808505,193.3,3100.0,0.808505,3100.0,200294.42,-1.0,-1.0,2.25,NaN,1.180889,0.0,2.25,2.25,3.10,0.506969,0.700000
9,IBULHSGFI,20201231,140.0,25,P,0.808505,193.3,3100.0,0.808505,3100.0,194598.16,-1.0,-1.0,1.35,NaN,1.303134,0.0,1.35,1.35,3.00,0.312485,0.700000


# Volatility

### Which symbols are the most volatile?

In [17]:
cols1 = ['symbol', 'undPrice', 'margin', 'high52week', 'low52week', 'histVolatility', 'iv']
df_unds.sort_values('iv', ascending=False)[cols1]

,symbol,undPrice,margin,high52week,low52week,histVolatility,iv
11,IDEA,9.95,680801.83,13.500000,2.700000,0.586931,0.920246
137,IBULHSGFI,193.30,601300.81,359.750000,81.000000,0.539990,0.808505
16,RBLBANK,231.80,616980.01,370.899994,101.550003,0.483442,0.623243
31,INDUSINDB,896.25,733600.00,1585.000000,235.550003,0.537925,0.592186
36,INFRATEL,239.55,662446.39,256.500000,113.650002,0.652534,0.574355
...,...,...,...,...,...,...,...
129,MARICO,389.65,788935.97,398.299988,231.000000,0.309443,0.268416
55,TCS,2709.45,817184.99,2873.000000,1471.050049,0.301576,0.267614
24,DABUR,501.50,627788.73,534.200012,382.700012,0.234822,0.257436
133,HINDUNILV,2127.75,656640.01,2576.800049,1719.800049,0.221435,0.246479


### Pick up a symbol

In [ ]:
symbol = 'RELIANCE'

In [ ]:
cols2 = ['symbol', 'strike', 'undPrice', 'dte', 'right', 'und_iv', 'bid', 'ask', 'close', 'last', 'margin', 'iv', 'intrinsic', 'timevalue', 'prob', 'sdMult', 'price', 'rom']
df = df_opts[df_opts.symbol == symbol][cols2]

In [ ]:
m1 = df_opts.symbol == symbol
m2 = df_opts.strike == 1800
df_opts[m1&m2][cols2]

In [ ]:
und_cols = ['symbol', 'low13week',	'high13week',	'low26week',	'high26week',	'low52week'	,'high52week', 'iv', 'undPrice', 'margin']
df_unds[df_unds.symbol == symbol][und_cols]

In [ ]:
# removes options without time value
m1 = ((df.right == 'C') & (df.strike > df.undPrice)) | \
    ((df.right == 'P') & (df.strike < df.undPrice))

# removes out-of-fence (sdMult) options
m2 = ((df.right == 'C') & (df.sdMult > ibp.CALLSTDMULT)) | \
    ((df.right == 'P') & (df.sdMult > ibp.PUTSTDMULT))

df = df[m1&m2].reset_index(drop=True)

### `rom` for the best `sdMult`

In [ ]:
# Sort down sdMult based on dte and right, groupby dte, right
s = df.sort_values(['dte', 'right', 'sdMult'], ascending=[True, False, True]).groupby(['symbol', 'dte', 'right']).cumcount()

# Filtered and Sorted df
df = df.loc[s.index]

# Option with the largest rom for the dte and right
df_max_rom = df[df.rom == df.groupby(['symbol', 'dte', 'right']).rom.transform(max)]

In [ ]:
df_max_rom.sort_values('rom', ascending=False)

In [ ]:
dfrq.sort_values('symbol').set_index('symbol').remq.to_dict()

## For `all` symbols
### Let us try this using sort, filter and groupby sequence

In [ ]:
# Read all opts (to reset from single opt above)
df = df_opts
cols2 = ['symbol', 'strike', 'undPrice', 'dte', 'right', 'und_iv', 'bid', 'ask', 'close', 'last', 'margin', 'iv', 'intrinsic', 'timevalue', 'prob', 'sdMult', 'price', 'rom']


# ...remove options without time value
m1 = ((df.right == 'C') & (df.strike > df.undPrice)) | \
    ((df.right == 'P') & (df.strike < df.undPrice))

# ...remove out-of-fence (sdMult) options
m2 = ((df.right == 'C') & (df.sdMult > ibp.CALLSTDMULT)) | \
    ((df.right == 'P') & (df.sdMult > ibp.PUTSTDMULT))

df = df[m1&m2].reset_index(drop=True)

# Sort down sdMult based on dte and right, groupby dte, right
s = df.sort_values(['dte', 'right', 'sdMult'], ascending=[True, False, True]).groupby(['symbol', 'dte', 'right']).cumcount()

# Filtered and Sorted df
df = df.loc[s.index]

# Option with the largest rom for the dte and right
df_max_rom = df[df.rom == df.groupby(['symbol', 'dte', 'right']).rom.transform(max)].sort_values('rom', ascending=False)

In [ ]:
len(df)

In [ ]:
df_max_rom[cols2]

In [ ]:
# * SORT
df1 = df_opts.sort_values(['symbol', 'dte', 'right', 'sdMult'], 
                          ascending=[True, True, False, True])

# * FILTER
# ... filter for options with timevalue
f1 = ((df1.right == 'C') & (df1.strike > df1.undPrice)) | \
    ((df1.right == 'P') & (df1.strike < df1.undPrice))

# ... filter for sdMult limits
f2 = ((df1.right == 'C') & (df1.sdMult > ibp.CALLSTDMULT)) | \
    ((df1.right == 'P') & (df1.sdMult > ibp.PUTSTDMULT))

df2 = df1[f1&f2].reset_index(drop=True)



In [ ]:
# Filter for options with timevalue
m1 = ((df_opts.right == 'C') & (df_opts.strike > df_opts.undPrice)) | \
    ((df_opts.right == 'P') & (df_opts.strike < df_opts.undPrice))

# Filter for sdMult limits
m2 = ((df_opts.right == 'C') & (df_opts.sdMult > ibp.CALLSTDMULT)) | \
    ((df_opts.right == 'P') & (df_opts.sdMult > ibp.PUTSTDMULT))


# groupby symbol
df1 = df_opts[m1&m2].sort_values(['dte', 'right', 'sdMult'], ascending=[True, False, True]).groupby('symbol').head(1)

### experiments with sort and groupby

In [ ]:
import random
import pandas as pd
random.seed(999)
sz = 50

qty = {'one': 1, 'two': 2, 'three': 3}

thing = (random.choice(['one', 'two', 'three']) for _ in range(sz))
order = (random.choice(['ascending', 'descending']) for _ in range(sz))
value = (random.randint(0, 100) for _ in range(sz))

df = pd.DataFrame({'thing': thing, 'order': order, 'value': value})

In [ ]:
df.loc[df.order=='descending', 'value']*= -1

In [ ]:
df.head()

In [ ]:
df.sort_values('value').groupby(['thing', 'order']).cumcount()

In [ ]:
SYMBOL = ''
EARLIEST = True

In [ ]:
if (SYMBOL != '') | EARLIEST:
    print(f'Something is there in {SYMBOL}, or earliest is {EARLIEST}')

In [ ]:
{df_symlots[df_symlots.symbol == 'RELIANCE'].contract.iloc[0]}